# Ensemble Methods and Pipelines

We could spend an entire course on ensemble methods.  Ensembles combine more than one model in some systematic way. In practice, data scientists do this because it helps them predict outcomes with more accuracy and with less bias than any single method does.  Because there are thousands of possible algorithms that can be combined in what are effectively limitless ways, there is no way to cover all of ensemble methods.  Instead, we will discuss one particular ensemble method, RANDOM FORESTS, that is very widely used.  You should be aware, however, that the idea of combining multiple relatively "weak" machine learning methods into a single "strong" machine learning method is quite common, and that most sophisticated machine learning models do this. 

Also in this assignment, we will introduce the idea of a "pipeline", a collection of steps that you can use to automate much of your data science process so that testing modifications and variations is relatively simple and straightforward.  Pipelines require a little getting used to, but they are worth the effort. 

Rather than run through a bunch of explanations, we're going to build an ensemble model, then talk about what it does. Then we'll do the same for a pipeline. 

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore') 
import numpy as np
import sklearn
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [2]:
# first let's read in the Titanic data 
df_train = pd.read_csv('train.csv', header = 0)
df_test = pd.read_csv('test.csv', header = 0)
df = pd.concat([df_train, df_test], keys=["train", "test"])

# then a little feature engineering
df['Age'] = df['Age'].fillna(df['Age'].median())
df['Child'] = df['Age'] < 14
df['Fare'] = df['Fare'].fillna(df['Fare'].median())

df['Title'] = df['Name'].map(lambda name:name.split(',')[1].split('.')[0].strip())  # titles might mean something
df['Title'][df.Title == 'Jonkheer'] = 'Master'
df['Title'][df.Title.isin(['Ms','Mlle'])] = 'Miss'
df['Title'][df.Title == 'Mme'] = 'Mrs'
df['Title'][df.Title.isin(['Capt', 'Don', 'Major', 'Col', 'Sir'])] = 'Sir'
df['Title'][df.Title.isin(['Dona', 'Lady', 'the Countess'])] = 'Lady'

df['NameLength'] = df.Name.apply(lambda x: len(x))   # fancy titles are longer
df = pd.get_dummies(data=df, columns=['Pclass','Sex','Embarked','Title'], drop_first=False).copy()  # make dummies for categories
df = df.drop(['Name','Cabin','Ticket'], axis=1).copy().dropna()  # drop the categorical columns
data = df.loc['train'] 
features = data.columns[1:]

In [3]:
# and then cross validation just like the last class with the rat data
cv = KFold(n_splits=5, shuffle=True, random_state=1)
for train_index, test_index in cv.split(data):
    X_train = data.loc[train_index].drop(['Survived'], axis=1)
    y_train = data.loc[train_index]['Survived']
    X_test = data.loc[test_index].drop(['Survived'], axis=1)
    y_test = data.loc[test_index]['Survived']

    clf = LogisticRegression()
    clf.fit(X_train, y_train)
    
    predicted = clf.predict(X_test)
    print('Precision: '+str(100 * round(precision_score(y_test, predicted),3)))

Precision: 78.1
Precision: 78.6
Precision: 81.5
Precision: 79.0
Precision: 74.6


# Random Forest Classifiers

Now copy that last block of code, and where you see 

```clf = LogisticRegression() ```

replace it with 

```clf = RandomForestClassifier()```

Compare your precision scores.  How do they compare? 

In [4]:
# And now let's do the exact same thing, but let's use a Random Forest Classifier
cv = KFold(n_splits=5, shuffle=True, random_state=1)
for train_index, test_index in cv.split(data):
    X_train = data.loc[train_index].drop(['Survived'], axis=1)
    y_train = data.loc[train_index]['Survived']
    X_test = data.loc[test_index].drop(['Survived'], axis=1)
    y_test = data.loc[test_index]['Survived']

    clf = RandomForestClassifier(random_state=1)
    clf.fit(X_train, y_train)
    
    predicted = clf.predict(X_test)
    print('Precision: '+str(100 * round(precision_score(y_test, predicted),3)))


Precision: 84.9
Precision: 83.6
Precision: 83.9
Precision: 79.6
Precision: 78.0


We changed the model we used from LogisticRegression to RandomForestClassifier. What is that, and why did that change increase the average precision?  

A logistic regression is an example of a linear model, which assumes that the relationship between each feature and the output class is a linear one. For example, if one of the features is the depth of a well, a linear model will assume that (all other things being equal) the difference in functionality between a 10-year-old and a 50-year-old  will be the same as the difference between 40-year-old and 90-year-old. This isn’t always a valid assumption. One way to address it is to add extra features like age squared and the logarithm of age, which helps a linear model capture nonlinearities, but might not still allow me to get all the nuances of nonlinear relationships.A logistic regression also doesn’t capture interactions between features. I can explicitly add interaction terms to the logistic regression, but this gets unwieldy fast when I have many features.

A decision tree can capture interactions and nonlinearities much more naturally than logistic regression, because of the binary tree structure of the decision tree algorithm itself. The downside of decision trees is that they can be harder to interpret or assign uncertainties to their predictions.

A random forest is a collection of decision trees, each of which is trained on a subset of the rows/columns of the training data. The randomness in the training set means that the individual trees in a random forest are high-variance, but low-bias, and the final prediction is made by having each tree classify a given event and then using their predictions as “votes,” with the majority opinion being assigned as the label. Nonlinearities and interactions are captured by the individual trees, but ensembling many trees into a random forest tends to cancel out the biases/shortcomings of any one tree producing a stronger predictor overall.

In empirical studies of many algorithms being applied to many supervised learning problems, random forests often come out on top overall. So when in doubt, or if I only have the time/resources to try one model, a random forest is likely to get at or near the peak performance of all the algorithms on the market.

My favorite explanation of random forests involves actual forests.  Have a look here: http://whrc.org/wp-content/uploads/2016/02/DecisionTrees_RandomForest_v2.pdf


## Random forest components

Random forests do not have a single canonical form.  Sci-Kit Learn has great defaults, but under the covers there are some important parameters that you may need to tune.  These include "max_features", 


### max_features

max_features sets the maximum number of features in an individual tree and is probably the most important parameter. The more features allowed, the more computations (and thus time) required. Also, the greater chance of overfitting.  Here are a few standard options for this parameter:

- Auto/None : This will simply take all the features which make sense in every tree.Here we simply do not put any restrictions on the individual tree.
- sqrt : This option will take square root of the total number of features in individual run. For instance, if the total number of variables are 100, we can only take 10 of them in individual tree.”log2″ is another similar type of option for max_features.
- 0.2 : This option allows the random forest to take 20% of variables in individual run. We can assign and value in a format “0.x” where we want x% of features to be considered.


### max_depth

max_depth sets the maximum  depth of the tree. By default, then nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples.  Because tree depth can be quite deep with lots of features, this adds another way to reduce overfitting and increase speed. 


### n_estimators 

n_estimators sets the number of trees built before each "weak learning" tree votes. More trees means more computation and thus more time. But this does not risk overfitting. You should choose as high value as your processor can handle because this makes your predictions stronger and more stable. sklearn defaults to ten.  


### min_sample_leaf

min_sample_leave (minimum sample leaf size) sets how many outcome votes have to exist for a leaf to have any way.  Smaller leaf makes the model more prone to overfitting. But if you don't have much data, it's best to start with the default (1).  If you think you are overfitting, you can increase that number.

### oob_score\
Finally, we can use a kind of internal cross validation special to random forests to estimate the generalization accuracy. This really helps reduce overfitting, but also cuts out potentially important edge cases your classifier might identify. 

To set a parameter, you just add it inside the parentheses after the model name.  Like so: 

```     clf = RandomForestClassifier(random_state=1, min_sample_leaf=2) ```

# excise 01 - Adjust Parameters

Re-run our random forest example from above, but this time set max_features to 20 and oob_score to 'True'. 

# Exercise 02 -- try  with your data

Now, try using the RandomForest classifier (if your dependent variable is binary or categorical) or regressor (if it is continuous) on your own data.  Start with the defaults (no parameter settings), then tweak some of them and see how they affect your model's speed and performance. 

# Pipelines

Pipelines help you automate some of your work and make it a little more systematic.  As you can probably tell, even though you aren't a software engineer, machine learning is all about letting machines do the work of learning.  

For example, we could add or subtract features, and see what happens, just like the old-timey statistians used to do.  But what if we have hudreds or thousands to choose from? And, if we decide to use random forests or some other modern ML model, we could tweak the parameters, like number of featurs, and max tree depth. But there are so many! 

Wouldn't it be much easiers to just let the computer do that? Plus, if we give the computer some generic standards, we can take our own biases out of feature selection and parameter tuning.  

That's what pipelines allow you to do.  You build a pipeline, and then tell the computer to use that pipeline to run lots of combinations of different features and parameters, and tell you what works best.  Let's try it out quickly, then explore a bit after that. 

## Pipelines

A pipeline just takes some of the steps in your model-builng process and automates it.  We'll build a simple one here, but they can get quite coplex.  In fact, the more complex your process, the more you'll probably want to use pipelines. 

Pipelines can do two things: transform data (tyically for feature engineering) and estimate with the data (predicting outcomes given some data).  

In [5]:
import sklearn.pipeline
X = data.drop(['Survived'], axis=1).as_matrix()
y = data['Survived'].as_matrix()

select = sklearn.feature_selection.SelectKBest(k='all')
clf = sklearn.ensemble.RandomForestClassifier(random_state=1)

steps = [('feature_selection', select),
         ('random_forest', clf)]

pipeline = sklearn.pipeline.Pipeline(steps) 
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, random_state=1)

pipeline.fit( X_train, y_train ) # fit your pipeline on X_train and y_train
y_prediction = pipeline.predict( X_test ) # call pipeline.predict() on your X_test data to make a set of test predictions
report = sklearn.metrics.classification_report( y_test, y_prediction ) # test your predictions using sklearn.classification_report()
print(report) # and print the report


             precision    recall  f1-score   support

        0.0       0.75      0.90      0.82       128
        1.0       0.81      0.60      0.69        95

avg / total       0.78      0.77      0.76       223



## Grid Search & Parameter Tuning

But where pipelines really shine is tuning hyperparameters.  You can do this with for loops, but pipelines make it much easiers.  

For exmple, with Random Forest Classification, we might want to tinker with parameters like the number of estimators, or the minimum number of samples.  Each of this is settable by using sklearn's pipeline and grid search tools.  The only trick is that we set these by using the model name, then two underscores, then the parameter name.   So instead of using

```
random_forest.n_estimators
```

you should use 

```
random_forest__n_estimators
```

We follow that with a list of the values we want to try.  So, if I wanted to try all the values between 5 and 10 I could use either: 

``` 
random_forest__n_estimators=[5,6,7,8,9, 10]
```

or

``` 
random_forest__n_estimators=list(range(5,11))
```

which produces the same thing.  If I think that trying every value will take too long (remember, every additional variation is multiplied by all the other variations!), then maybe just try a few.  

So here is how we search over a large number of possible parameters: 

In [6]:
import sklearn.grid_search

parameters = dict(feature_selection__k=[5, 7, 9, 11, 13, 15, 17, 19, 20], # listing each one
              random_forest__n_estimators=list(range(5,21,2)),  # generating a list up, skipping by two
              random_forest__min_samples_split=list(range(2,11,2))) 

cv = sklearn.grid_search.GridSearchCV(pipeline, param_grid=parameters, verbose=True)

cv.fit(X_train, y_train)
y_predictions = cv.predict(X_test)
report = sklearn.metrics.classification_report(y_test, y_predictions)
print(report)

Fitting 3 folds for each of 360 candidates, totalling 1080 fits


/opt/conda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


             precision    recall  f1-score   support

        0.0       0.77      0.92      0.84       128
        1.0       0.86      0.63      0.73        95

avg / total       0.81      0.80      0.79       223



[Parallel(n_jobs=1)]: Done 1080 out of 1080 | elapsed:   23.1s finished


So our pipeline tried 1080 possible parameter combinations, and it fit across all of them.  Sure, you could do that by hand, but ... why would you?  Pipelines are great for this kind of extremely tedious work. Whenever possible, just let the computer do it!

# exercise 03 - try with your own data

Now try a grid search over a few parameters using your own data.  Start small -- it can take a while!  

# Expanding your horizons

We don't have time to cover all the possible classifiers you can possibly use, but in the code that follows, we run through a few that you might find useful: 

In [7]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

names = ["Nearest Neighbors", "Linear SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]

classifiers = [
    KNeighborsClassifier(),
    SVC(),
    GaussianProcessClassifier(),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(),
    MLPClassifier(),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]


for name, clf in zip(names, classifiers):
    cv = KFold(n_splits=5, shuffle=True, random_state=1)
    for train_index, test_index in cv.split(data):
        X_train = data.loc[train_index].drop(['Survived'], axis=1)
        y_train = data.loc[train_index]['Survived']
        X_test = data.loc[test_index].drop(['Survived'], axis=1)
        y_test = data.loc[test_index]['Survived']

        clf.fit(X_train, y_train)
        score = clf.score(X_test, y_test)
        print(name, score)

Nearest Neighbors 0.653631284916
Nearest Neighbors 0.662921348315
Nearest Neighbors 0.629213483146
Nearest Neighbors 0.634831460674
Nearest Neighbors 0.61797752809
Linear SVM 0.59217877095
Linear SVM 0.556179775281
Linear SVM 0.606741573034
Linear SVM 0.623595505618
Linear SVM 0.64606741573
Gaussian Process 0.597765363128
Gaussian Process 0.601123595506
Gaussian Process 0.550561797753
Gaussian Process 0.573033707865
Gaussian Process 0.634831460674
Decision Tree 0.776536312849
Decision Tree 0.752808988764
Decision Tree 0.831460674157
Decision Tree 0.825842696629
Decision Tree 0.831460674157
Random Forest 0.765363128492
Random Forest 0.792134831461
Random Forest 0.842696629213
Random Forest 0.859550561798
Random Forest 0.831460674157
Neural Net 0.776536312849
Neural Net 0.769662921348
Neural Net 0.589887640449
Neural Net 0.674157303371
Neural Net 0.842696629213
AdaBoost 0.776536312849
AdaBoost 0.792134831461
AdaBoost 0.820224719101
AdaBoost 0.820224719101
AdaBoost 0.820224719101
Naive Ba

# exercise 04 - try with your own data

Try another algorithm with your own data. Sklean makes it easy!